In [1]:
!apt-get update
!apt-get install -q -y curl systemctl unzip

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2377 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy/main amd64 Packages [1792 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy/restricted amd64 Packages [164 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [17.5 MB] 
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3205 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 Packages [266 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1162 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1450 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy

In [5]:
# 下載 gguf 到 /tmp
# TODO 修改為要使用的GGUF
!cd /tmp && { curl -OL https://huggingface.co/bartowski/Qwen2.5-14B-Instruct-GGUF/resolve/main/Qwen2.5-14B-Instruct-Q4_K_M.gguf ;}

# 安裝 ollama
!curl -L https://ollama.com/download/ollama-linux-amd64.tgz -o ollama-linux-amd64.tgz
!tar -C /usr -xzf ollama-linux-amd64.tgz
!useradd -r -s /bin/false -U -m -d /usr/share/ollama ollama
!usermod -a -G ollama $(whoami)
# 建立 ollama service
content = """[Unit]
Description=Ollama Service
After=network.target

[Service]
ExecStart=/usr/bin/ollama start
User=ollama
WorkingDirectory=/usr/share/ollama
Restart=always

[Install]
WantedBy=multi-user.target
"""
file_path = '/tmp/ollama.service'
with open(file_path, 'w') as file:
    file.write(content)
!cp /tmp/ollama.service /etc/systemd/system/ollama.service
!systemctl start ollama

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1148  100  1148    0     0  11640      0 --:--:-- --:--:-- --:--:-- 11714
100 8571M  100 8571M    0     0  34.6M      0  0:04:07  0:04:07 --:--:-- 40.5M   1  109M    0     0  30.0M      0  0:04:45  0:00:03  0:04:42 32.8M571M   44 3792M    0     0  30.0M      0  0:04:45  0:02:06  0:02:39 40.7M55 4784M    0     0  31.3M      0  0:04:33  0:02:32  0:02:01 32.7M 4820M    0     0  31.4M      0  0:04:32  0:02:33  0:01:59 34.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1586M  100 158

In [19]:
# 把GGUF加入ollama中
# TODO 如有更換其他GGUF要修改檔案名稱
content = """FROM /tmp/Qwen2.5-14B-Instruct-Q4_K_M.gguf
TEMPLATE "{{ if .System }}<|im_start|>system
{{ .System }}<|im_end|>
{{ end }}{{ if .Prompt }}<|im_start|>user
{{ .Prompt }}<|im_end|>
{{ end }}<|im_start|>assistant
{{ .Response }}<|im_end|>"
PARAMETER num_ctx 8192
PARAMETER stop "<|im_end|>"
PARAMETER stop "<|im_start|>"
"""
# Write the content to the file
file_path = '/tmp/Modelfile'
with open(file_path, 'w') as file:
    file.write(content)
# TODO 這個Qwen2.5-14B-Instruct-Q4_K_M模型名稱可自行填寫 將用於後續使用api呼叫時填寫的模型名
!ollama create Qwen2.5-14B-Instruct-Q4_K_M -f /tmp/Modelfile

transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠼ transferring model data ⠦ transferring model data ⠧ transferring model data ⠇ transferring model data ⠇ transferring model data ⠋ transferring model data ⠙ transferring model data ⠙ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠦ transferring model data ⠧ transferring model data ⠇ transferring model data ⠏ transferring model data ⠏ transferring model data ⠙ transferring model data ⠹ transferring model data ⠹ transferring model data ⠼ transferring model data ⠴ transferring model data ⠦ transferring model data ⠧ transferring model data ⠧ transferring model data ⠏ transferring model data ⠋ transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠦ transferring model data ⠧ transferring model data ⠇ transferring

In [15]:
# 安裝 ngrok
!mkdir -p /tmp/ngrok-ssh
!cd /tmp/ngrok-ssh
!curl -OL https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /tmp/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok
!pip install pyngrok

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.2M  100 13.2M    0     0  37.1M      0 --:--:-- --:--:-- --:--:-- 37.1M
Archive:  ngrok-stable-linux-amd64.zip


In [ ]:
# TODO 修改ngrok的authtoken
!ngrok config add-authtoken 2o...{your-authtoken}...aV
!echo 'debconf debconf/frontend select Noninteractive' | debconf-set-selections
from pyngrok import ngrok
# TODO 修改ngrok的authtoken
ngrok.set_auth_token("2o...{your-authtoken}...aV")

import os
import asyncio

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )

await asyncio.gather(
    run_process(['ollama', 'serve']),
    run_process(['ngrok', 'http', '11434', '--log', 'stderr', '--host-header', 'localhost:11434']),
)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
>>> starting ngrok http 11434 --log stderr --host-header localhost:11434
t=2024-10-29T07:07:23+0000 lvl=info msg="no configuration paths supplied"
t=2024-10-29T07:07:23+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
t=2024-10-29T07:07:23+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
t=2024-10-29T07:07:23+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
t=2024-10-29T07:07:23+0000 lvl=info msg="client session established" obj=tunnels.session
t=2024-10-29T07:07:23+0000 lvl=info msg="tunnel session started" obj=tunnels.session
t=2024-10-29T07:07:23+0000 lvl=info msg="started tunnel" obj=tunnels name=command_line addr=http://localhost:11434 url=https://8784-3-136-26-80.ngrok-free.app
t=2024-10-29T07:08:27+0000 lvl=info msg="join connections" obj=join id=f66b7437a962 l=127.0.0.1:11434 r=36.238.178.211:7603